In [7]:
import numpy as np
import pandas as pd
from collections import defaultdict
import operator
from utils import *

https://pslcdatashop.web.cmu.edu/KDDCup/downloads.jsp

In [3]:
bath_path = '/mnt/qb/work/mlcolab/hzhou52/kt/junyii/'

# exercise 837, topic 41, area 9
exercise = pd.read_csv(bath_path+'junyi_Exercise_table.csv', encoding = "utf-8",low_memory=False) 
relation = pd.read_csv(bath_path+'relationship_annotation_training.csv', encoding = "utf-8",low_memory=False)
log = pd.read_csv(bath_path+'junyi_ProblemLog_original.csv', encoding = "utf-8",low_memory=False)

In [5]:
log.head()

,user_id,exercise,problem_type,problem_number,topic_mode,suggested,review_mode,time_done,time_taken,time_taken_attempts,correct,count_attempts,hint_used,count_hints,hint_time_taken_list,earned_proficiency,points_earned
0,12884,time_terminology,analog_word,1,False,False,False,1420714810324490,4,3&1,False,2,False,0,NaN,False,0
1,239464,multiplication_1,0,6,False,False,False,1403098400836660,2,2,True,1,False,0,NaN,False,14
2,147359,adding_decimals_0.5,0,6,False,False,False,1418890695540340,16,16,True,1,False,0,NaN,False,75
3,158155,multiplication_1,0,3,False,False,False,1400469444264040,2,2,True,1,False,0,NaN,False,75
4,147151,subtraction_2,subtraction-2,10,True,True,False,1382650905730160,4,4,True,1,False,0,NaN,False,225


In [8]:
check_nan(log)
count_unique(log, ['user_id', 'exercise'])
print(len(log))

number of NaN in column user_id: 0
number of NaN in column exercise: 0
number of NaN in column problem_type: 0
number of NaN in column problem_number: 0
number of NaN in column topic_mode: 0
number of NaN in column suggested: 0
number of NaN in column review_mode: 0
number of NaN in column time_done: 0
number of NaN in column time_taken: 0
number of NaN in column time_taken_attempts: 62365
number of NaN in column correct: 0
number of NaN in column count_attempts: 0
number of NaN in column hint_used: 0
number of NaN in column count_hints: 0
number of NaN in column hint_time_taken_list: 24254263
number of NaN in column earned_proficiency: 0
number of NaN in column points_earned: 0
number of unique value in user_id: 247606
number of unique value in exercise: 722
25925992


In [9]:
log = remove_log(log, 'user_id')
print(len(log))

25886525


# pre-process

In [10]:
# create new data
df = pd.DataFrame(columns=['user_id', 'problem_id', 'skill_id', 'timestamp', 'dwell_time', 'correct'])
df.user_id = log.user_id
df.problem_id = log.exercise
df.skill_id = log.exercise
df.timestamp = log.time_done
df.dwell_time = log.time_taken
df.correct = log.correct * 1

In [13]:
df

,user_id,problem_id,skill_id,timestamp,dwell_time,correct
0,12884,time_terminology,time_terminology,1420714810324490,4,0
1,239464,multiplication_1,multiplication_1,1403098400836660,2,1
2,147359,adding_decimals_0.5,adding_decimals_0.5,1418890695540340,16,1
3,158155,multiplication_1,multiplication_1,1400469444264040,2,1
4,147151,subtraction_2,subtraction_2,1382650905730160,4,1
...,...,...,...,...,...,...
25925987,114872,equivalent_fractions,equivalent_fractions,1377638408722100,7,1
25925988,79371,dividing_decimals_0.5,dividing_decimals_0.5,1402970909936040,34,1
25925989,242580,telling_time,telling_time,1397029145893100,18,1
25925990,113514,subtraction_1,subtraction_1,1396568072096380,2,1


In [14]:
# re-index
# - skill
skill_id = list(df['skill_id'].unique())
skill_dict = dict(zip(skill_id, range(len(skill_id))))
df['skill_id'] = df['skill_id'].apply(lambda x: skill_dict[x])

# - problem
df['problem_id'] = df['skill_id']
# pro_id = list(df['problem_id'].unique())
# pro_dict = dict(zip(pro_id, range(len(pro_id))))
# df['problem_id'] = df['problem_id'].apply(lambda x: pro_dict[x])

# - student
stu_id = list(df['user_id'].unique())
stu_dict = dict(zip(stu_id, range(len(stu_id))))
df['user_id'] = df['user_id'].apply(lambda x: stu_dict[x])

In [16]:
df

,user_id,problem_id,skill_id,timestamp,dwell_time,correct
0,0,0,0,1420714810324490,4,0
1,1,1,1,1403098400836660,2,1
2,2,2,2,1418890695540340,16,1
3,3,1,1,1400469444264040,2,1
4,4,3,3,1382650905730160,4,1
...,...,...,...,...,...,...
25925987,18886,136,136,1377638408722100,7,1
25925988,37547,168,168,1402970909936040,34,1
25925989,38629,87,87,1397029145893100,18,1
25925990,17489,31,31,1396568072096380,2,1


In [17]:
df = df.astype({
    'timestamp': np.float64,
    'dwell_time': np.float64,
    'correct': np.float64,
    'problem_id': np.int64,
    'skill_id': np.int64
    })

In [18]:
# Save
df.to_csv(bath_path+'interactions.csv', sep='\t', index=False)

In [19]:
count_unique(df, list(df.columns))
len(df)

number of unique value in user_id: 208139
number of unique value in problem_id: 722
number of unique value in skill_id: 722
number of unique value in timestamp: 25884747
number of unique value in dwell_time: 7780
number of unique value in correct: 2


25886525